# 12 Steps to Naiver-Stokes

The final two steps in this interactive module teaching beginning CFD with Python will both solve the Naiver-Stokes equations in two dimensions, but with different boundary conditions.

The momentum equation in vector form for a velocity field $\vec{v}$ is :
$$
\dfrac{\partial{\vec{v}}}{\partial{t}}+\big(\vec{v}\cdot\nabla\big)\vec{v} = -\dfrac{1}{\rho}\nabla{p}+\nu\nabla^2\vec{v}
$$

This represents three scalar equations, one for each velocity component$(u, v, w)$. But we will solve it in two dimensions, so where will be two scalar equations.

Remember the continuity equation? This is where in Poisson equation for pressure comes in!

## Step11: Cavity Flow with Navier-Stokes

Here is the system of differential equations: two equations for the velocity components $(u, v)$ and one equation for pressure:

$$
\dfrac{\partial{u}}{\partial{t}}+u\dfrac{\partial{u}}{\partial{x}}+v\dfrac{\partial{u}}{\partial{y}}=-\dfrac{1}{\rho}\dfrac{\partial{p}}{\partial{x}}+\nu\big(\dfrac{\partial^2{u}}{\partial{x^2}}+\dfrac{\partial^2{u}}{\partial{y}^2}\big) \\
\dfrac{\partial{v}}{\partial{t}}+u\dfrac{\partial{v}}{\partial{x}}+
v\dfrac{\partial{v}}{\partial{y}}=-\dfrac{1}{\rho}\dfrac{\partial{p}}{\partial{y}}+\nu\big(\dfrac{\partial^2{v}}{\partial{x^2}}+\dfrac{\partial^2{v}}{\partial{y^2}}\big) \\
\dfrac{\partial^2{p}}{\partial{x^2}}+\dfrac{\partial^2{p}}{\partial{y^2}}=-\rho\big(\dfrac{\partial{u}}{\partial{x}}\dfrac{\partial{u}}{\partial{x}} + 2\dfrac{\partial{u}}{\partial{y}}\dfrac{\partial{v}}{\partial{x}} + \dfrac{\partial{v}}{\partial{y}}\dfrac{\partial{v}}{\partial{y}}\big)+\rho\dfrac{\partial}{\partial{t}}\big(\dfrac{\partial{u}}{\partial{x}}+\dfrac{\partial{u}}{\partial{y}}\big)
$$

Frome the previous steps, we already know how to discretize all these terms. Only the last equation is a little unfamiliar. But with a little patience, it will not be hard!

## Discretized equations

First, let's discretize the $u$-momentum equation, as follows:

$$
\dfrac{u_{i,j}^{n+1} - u_{i,j}^{n}}{\Delta{t}}+
u_{i,j}^{n}\dfrac{u_{i,j}^{n}-u_{i-1,j}^{n}}{\Delta{x}}+
v_{i,j}^{n}\dfrac{u_{i,j}^{n}-u_{i,j-1}^{n}}{\Delta{y}}=-\dfrac{1}{\rho}
\dfrac{p_{i+1,j}^{n}-p_{i-1,j}^{n}}{2\Delta{x}}+\nu\big(
\dfrac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\Delta{x^2}}+
\dfrac{u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}}{\Delta{y^2}}\big)
$$

Similarly for the $v$-momentum equation:

$$
\dfrac{v_{i,j}^{n+1} - v_{i,j}^{n}}{\Delta{t}}+
u_{i,j}^{n}\dfrac{v_{i,j}^{n}-v_{i-1,j}^{n}}{\Delta{x}}+
v_{i,j}^{n}\dfrac{v_{i,j}^{n}-v_{i,j-1}^{n}}{\Delta{y}}=-\dfrac{1}{\rho}
\dfrac{p_{i+1,j}^{n}-p_{i-1,j}^{n}}{2\Delta{x}}+\nu\big(
\dfrac{v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}}{\Delta{x^2}}+
\dfrac{v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}}{\Delta{y^2}}\big)
$$

Finally, the disretized pressure-Poisson equation can be writtend thus:

$$
\dfrac{p_{i+1,j}^{n}-2p_{i,j}^{n}+p_{i-1,j}^{n}}{\Delta{x^2}}+
\dfrac{p_{i,j+1}^{n}-2p_{i,j}^{n}+p_{i,j-1}^{n}}{\Delta{y^2}}= \\
\rho
[\dfrac{1}{\Delta{t}}\big(\dfrac{u_{i+1,j}^{n}-u_{i-1,j}^{n}}{2\Delta{x}}+\dfrac{u_{i,j+1}^{n}-u_{i,j-1}^{n}}{2\Delta{y}}\big)]-\rho
\big(\dfrac{u_{i+1,j}^{n}-u_{i-1,j}^{n}}{2\Delta{x}}\dfrac{u_{i+1,j}^{n}-u_{i-1,j}^{n}}{2\Delta{x}}+
2\dfrac{u_{i,j+1}^{n}-u_{i,j-1}^{n}}{2\Delta{y}}\dfrac{v_{i+1,j}^{n}-v_{i-1,j}^{n}}{2\Delta{x}}+
\dfrac{v_{i,j+1}^{n}-v_{i,j-1}^{n}}{2\Delta{y}}
\dfrac{v_{i,j+1}^{n}-v_{i,j-1}^{n}}{2\Delta{y}}\big)
$$

**这个方程是我写的最头疼的**

$\LaTeX$敲死我了。关键教程11步错误太多了。要注意。

The initial condition is $u,v,p=0$ everwhere, and the boundary conditions are:

$u = 1 \ at \ y = 2(the "lid")$

$u = 0, v = 0 \ at \ x = 0, 2  y = 0$

$\frac{\partial{p}}{\partial{y}}=0 \ at y = 0$

$\frac{\partial{p}}{\partial{x}}=0 \ at x = 0, 2$

$p = 0 \ at \ y = 2$





## Implementing Cavity Flow

In [2]:
import numpy as np
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [3]:
nx = 41
ny = 41
dx = 2/(nx-1)
dy = 2/(ny-1)
nt = 500
nit = 50
rho = 1
nu = 0.1
dt = 0.001
x = np.linspace(0, 2, nx)
y = np.linspace(0, 2, ny)
X, Y = np.meshgrid(x,y)
u = np.zeros((ny, nx))
v = np.zeros((ny, nx))
p = np.zeros((ny, nx))
b = np.zeros((ny, nx))

In [8]:
def build_up_b(b, rho, dt, dx, dy, u):
    b[1:-1, 1:-1] = rho * (u[1:-1, 2:] - u[1:-1, :-2]) / (dt * 2 * dx)\
                  + rho * (u[2:, 1:-1] - u[-2:, 1:-1]) / (dt * 2 * dx)\
                  - rho *((u[1:-1, 2:] - u[1:-1, :-2]) / (2*dx)) ** 2 \
                  - rho *((v[2:, 1:-1] - v[:-2, 1:-1]) / (2*dy)) ** 2 \
                  - rho *((u[2:, 1:-1] - u[:-2, 1:-1])*((v[1:-1, 2:] - v[1:-1, :-2])) / (4*dx*dy))
    return b

In [10]:
def pressure_poisson(p, dx, dy, b):
    #define a pseudo-time variable-nit;
    for i in range(nit):
        pn = p.copy()
        p[1:-1, 1:-1] = ((dy**2)*(pn[1:1, 2:] - pn[1:1, -2]) + (dx ** 2)*(pn[2:,1:-1] - pn[:-2, 1:-1]))/(2*(dx**2 + dy**2))\
                  + (dx**2 * dy**2) * b /(2*(dx**2 + dy**2))
    
        p[:, -1] = p[:, -2]
        p[:, 0]  = p[:, 1]
        p[-1, :] = 0
        p[0, :] = p[1, :]
    return p

In [ ]:
def cavity_flow(nt, u, v, p, rho, nu, dt, dx, dy):
    for i in range(nt):
        un = u.copy()
        vn = v.copy()
        
        b = build_up_b(b, rho, dt, u, v, dx, dy)
        p = pressure_poisson(b, p, dx, dy)
        
        u[1:-1, 1:-1] = u[]